In [1]:
import json
import os
import pandas as pd
license_key = "/home/jovyan/work/shared/ahmet/keys/4.2.8.json"
with open(license_key) as f:
    license_keys = json.load(f)
    
locals().update(license_keys)

# Adding license key-value pairs to environment variables
os.environ.update(license_keys)

import sparknlp_jsl
import sparknlp
from pyspark.ml import Pipeline,PipelineModel
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.base import *

params = {'spark.jsl.settings.pretrained.cache_folder': '/home/jovyan/work/shared/cache_folder',
          'spark.settings.pretrained.cache_folder': '/home/jovyan/work/shared/cache_folder',
          'spark.executor.cores':10,
          "spark.driver.memory":"16G", 
          "spark.kryoserializer.buffer.max":"2000M", 
          "spark.driver.maxResultSize":"2048M"
         }

spark = sparknlp_jsl.start(secret=SECRET, params=params)
spark

:: loading settings :: url = jar:file:/home/jovyan/work/persistent/.local/share/jupyter/venvs/aet-base-work/lib/python3.8/site-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-48d727e2-cab6-4de3-b1b4-89129e739db2;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;4.2.8 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.828 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.projectlombok#lombok;1.16.8 in central
	found com.google.cloud#google-cloud-storage;2.15.0 in central
	found com.google.guava#guava;31.1-jre in central
	found com.google.guava#failur

In [2]:
text_1= """He is discharged in September. 
His pps number is 1234567. 
"""

df= spark.createDataFrame([[text_1]]).toDF("text")

In [3]:
df_pd= df.toPandas()
df_pd.head()

,text
0,He is discharged in September. \nHis pps numbe...


In [4]:
df_pd.to_csv("/home/jovyan/work/shared/ahmet/works/DEID_MODULE/try_data.csv", index=False)

# Example custom pipeline

In [2]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentenceDetector = SentenceDetector()\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

tokenizer = Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")

deid_ner = MedicalNerModel.pretrained("ner_deid_generic_augmented", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner")

ner_converter = NerConverterInternal()\
    .setInputCols(["sentence", "token", "ner"])\
    .setOutputCol("ner_chunk")\
    .setWhiteList(['NAME', 'PROFESSION', 'ID', 'AGE', 'DATE'])

nlpPipeline = Pipeline(stages=[
      documentAssembler, 
      sentenceDetector,
      tokenizer,
      word_embeddings,
      deid_ner,
      ner_converter])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model= nlpPipeline.fit(empty_data)

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ]embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
Download done! Loading the resource.
[ / ]

[OK!]
ner_deid_generic_augmented download started this may take some time.
[ | ]ner_deid_generic_augmented download started this may take some time.
Approximate size to download 13.8 MB
Download done! Loading the resource.
[ / ]

2023-01-27 10:51:58.193354: I external/org_tensorflow/tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-27 10:51:58.359204: W external/org_tensorflow/tensorflow/core/common_runtime/colocation_graph.cc:1218] Failed to place the graph without changing the devices of some resources. Some of the operations (that had to be colocated with resource generating operations) are not supported on the resources' devices. Current candidate devices are [
  /job:localhost/replica:0/task:0/device:CPU:0].
See below for details of this colocation group:
Colocation Debug Info:
Colocation group had the following types and supported devices: 
Root Member(assigned_device_name_index_=-1 requested_device_name_='/device:GPU:0' assigned_device_nam

[OK!]


## With all default values 
No custom pipeline <br/>
**Default:mask with entity_labels**

In [3]:
from new_oop import Deidentifier

In [4]:
deid_implementor= Deidentifier(spark)

In [5]:
res= deid_implementor.deidentify(input_file_path="/home/jovyan/work/shared/ahmet/works/DEID_MODULE/try_data.csv",
                                 output_file_path="deidentified1.csv")

clinical_deidentification download started this may take some time.
Approx size to download 1.6 GB
[ | ]clinical_deidentification download started this may take some time.
Approximate size to download 1.6 GB
Download done! Loading the resource.
[ / ]

2023-01-27 10:52:29.214031: W external/org_tensorflow/tensorflow/core/common_runtime/colocation_graph.cc:1218] Failed to place the graph without changing the devices of some resources. Some of the operations (that had to be colocated with resource generating operations) are not supported on the resources' devices. Current candidate devices are [
  /job:localhost/replica:0/task:0/device:CPU:0].
See below for details of this colocation group:
Colocation Debug Info:
Colocation group had the following types and supported devices: 
Root Member(assigned_device_name_index_=-1 requested_device_name_='/device:GPU:0' assigned_device_name_='' resource_device_name_='/device:GPU:0' supported_device_types_=[CPU] possible_devices_=[]
ScatterAdd: CPU 
RealDiv: CPU 
VariableV2: CPU 
Identity: CPU 
Add: CPU 
AddV2: CPU 
Unique: CPU 
Shape: CPU 
Sqrt: CPU 
AssignSub: CPU 
Sub: CPU 
Const: CPU 
Fill: CPU 
RandomUniform: CPU 
Mul: CPU 
Assign: CPU 
NoOp: CPU 
GatherV2: CPU 
UnsortedSegmentSum: CPU 
Cast: C

[ — ]

2023-01-27 10:52:30.733566: W external/org_tensorflow/tensorflow/core/common_runtime/colocation_graph.cc:1218] Failed to place the graph without changing the devices of some resources. Some of the operations (that had to be colocated with resource generating operations) are not supported on the resources' devices. Current candidate devices are [
  /job:localhost/replica:0/task:0/device:CPU:0].
See below for details of this colocation group:
Colocation Debug Info:
Colocation group had the following types and supported devices: 
Root Member(assigned_device_name_index_=-1 requested_device_name_='/device:GPU:0' assigned_device_name_='' resource_device_name_='/device:GPU:0' supported_device_types_=[CPU] possible_devices_=[]
ScatterAdd: CPU 
RealDiv: CPU 
VariableV2: CPU 
Identity: CPU 
Add: CPU 
AddV2: CPU 
Unique: CPU 
Shape: CPU 
Sqrt: CPU 
AssignSub: CPU 
Sub: CPU 
Const: CPU 
Fill: CPU 
RandomUniform: CPU 
Mul: CPU 
Assign: CPU 
NoOp: CPU 
GatherV2: CPU 
UnsortedSegmentSum: CPU 
Cast: C

[OK!]
Deidentification process of 'text' field is starting...


deidentification process of 'text' field is completed...
Deidentifcation successfully completed and the results saved as 'deidentified1.csv' !


In [9]:
res.show(truncate=False)

+---+-------------------------------+-----------------------------+
|ID |text_original                  |text_deid                    |
+---+-------------------------------+-----------------------------+
|0  |He is discharged in September. |[He is discharged in <DATE>.]|
|1  |His pps number is 1234567.     |[His pps number is <IDNUM>.] |
|2  |"                              |["]                          |
+---+-------------------------------+-----------------------------+



## Mask options 


In [6]:
from new_oop import Deidentifier
deid_implementor= Deidentifier(spark)

In [7]:
res= deid_implementor.deidentify(input_file_path="/home/jovyan/work/shared/ahmet/works/DEID_MODULE/try_data.csv",
                                output_file_path="deidentified1.csv",
                                custom_pipeline=model,
                                mode="mask", masking_policy="same_length_chars")

Deidentification process of 'text' field is starting...


Deidentification process of 'text' field is completed...
Deidentifcation successfully completed and the results saved as 'deidentified1.csv' !


In [8]:
res.show(truncate=False)

+---+-------------------------------+------------------------------+
|ID |text                           |text_deidentified             |
+---+-------------------------------+------------------------------+
|0  |He is discharged in September. |He is discharged in [*******].|
|1  |His pps number is 1234567.     |His pps number is [*****].    |
|2  |"                              |"                             |
+---+-------------------------------+------------------------------+



In [10]:
#checking saved output file
import pandas as pd
res_data= pd.read_csv("/home/jovyan/work/shared/ahmet/works/DEID_MODULE/new_version/deidentified1.csv")
res_data.head()

,ID,text,text_deidentified
0,0,He is discharged in September.,He is discharged in [*******].
1,1,His pps number is 1234567.,His pps number is [*****].
2,2,"""",""""


### fixed_length_chars

In [11]:
from new_oop import Deidentifier
deid_implementor= Deidentifier(spark)

In [12]:
res= deid_implementor.deidentify(input_file_path="/home/jovyan/work/shared/ahmet/works/DEID_MODULE/try_data.csv",
                                output_file_path="deidentified1.csv",
                                custom_pipeline=model,
                                mode="mask", masking_policy="fixed_length_chars", fixed_mask_length=2)

Deidentification process of 'text' field is starting...
Deidentification process of 'text' field is completed...
Deidentifcation successfully completed and the results saved as 'deidentified1.csv' !


In [13]:
res.show(truncate=False)

+---+-------------------------------+-----------------------+
|ID |text                           |text_deidentified      |
+---+-------------------------------+-----------------------+
|0  |He is discharged in September. |He is discharged in **.|
|1  |His pps number is 1234567.     |His pps number is **.  |
|2  |"                              |"                      |
+---+-------------------------------+-----------------------+



In [14]:
#checking saved output file
import pandas as pd
res_data= pd.read_csv("/home/jovyan/work/shared/ahmet/works/DEID_MODULE/new_version/deidentified1.csv")
res_data.head()

,ID,text,text_deidentified
0,0,He is discharged in September.,He is discharged in **.
1,1,His pps number is 1234567.,His pps number is **.
2,2,"""",""""


### masking multiple columns

In [15]:
text_1= """He is discharged in May. 
His names is John"""

text= """Name: Patrick Age: 25"""

df= spark.createDataFrame([[text, text_1]]).toDF("text", "text1")
df_pd= df.toPandas()
df_pd.to_csv("/home/jovyan/work/shared/ahmet/works/DEID_MODULE/pp_try_data.csv", index=False)
df_pd.head()

,text,text1
0,Name: Patrick Age: 25,He is discharged in May. \nHis names is John


In [16]:
res= deid_implementor.deidentify(input_file_path="/home/jovyan/work/shared/ahmet/works/DEID_MODULE/pp_try_data.csv",
                                output_file_path="deidentified1.csv",
                                custom_pipeline=model,
                                mode="mask", masking_policy="fixed_length_chars", fixed_mask_length=2, 
                                field_names=["text", "text1"], separator=",")

Deidentification process of 'text' field is starting...
Deidentification process of 'text' field is completed...
Deidentification process of 'text1' field is starting...
Deidentification process of 'text1' field is completed...
Deidentifcation successfully completed and the results saved as 'deidentified1.csv' !


In [17]:
res.show(truncate=False)

+---+---------------------+-----------------+-------------------------+-----------------------+
|ID |text                 |text_deidentified|text1                    |text1_deidentified     |
+---+---------------------+-----------------+-------------------------+-----------------------+
|0  |Name: Patrick Age: 25|Name: ** Age: ** |He is discharged in May. |He is discharged in **.|
|1  |His names is John"   |His names is **" |                         |NONE                   |
+---+---------------------+-----------------+-------------------------+-----------------------+



### Obfuscate Options

In [18]:
from new_oop import Deidentifier
deid_implementor= Deidentifier(spark)

In [19]:
res= deid_implementor.deidentify(input_file_path="/home/jovyan/work/shared/ahmet/works/DEID_MODULE/try_data.csv",
                                output_file_path="deidentified1.csv",
                                custom_pipeline=model,
                                mode="obfuscate", obfuscate_ref_source="file",
                                obfuscate_ref_file_path="/home/jovyan/work/shared/ahmet/works/DEID_MODULE/obfuscation.txt")

Deidentification process of 'text' field is starting...
Deidentification process of 'text' field is completed...
Deidentifcation successfully completed and the results saved as 'deidentified1.csv' !


In [20]:
res.show(truncate=False)

+---+-------------------------------+-------------------------------+
|ID |text                           |text_deidentified              |
+---+-------------------------------+-------------------------------+
|0  |He is discharged in September. |He is discharged in 05-12-1990.|
|1  |His pps number is 1234567.     |His pps number is <ID>.        |
|2  |"                              |"                              |
+---+-------------------------------+-------------------------------+



### obfuscate_ref_source=both

In [21]:
res= deid_implementor.deidentify(input_file_path="/home/jovyan/work/shared/ahmet/works/DEID_MODULE/try_data.csv",
                                output_file_path="deidentified1.csv",
                                custom_pipeline=model,
                                mode="obfuscate", obfuscate_ref_source="both",
                                obfuscate_ref_file_path="/home/jovyan/work/shared/ahmet/works/DEID_MODULE/obfuscation.txt")

Deidentification process of 'text' field is starting...


Deidentification process of 'text' field is completed...
Deidentifcation successfully completed and the results saved as 'deidentified1.csv' !


In [22]:
res.show(truncate=False)

+---+-------------------------------+-------------------------------+
|ID |text                           |text_deidentified              |
+---+-------------------------------+-------------------------------+
|0  |He is discharged in September. |He is discharged in 01-20-1979.|
|1  |His pps number is 1234567.     |His pps number is O7566230.    |
|2  |"                              |"                              |
+---+-------------------------------+-------------------------------+



### obfuscate_ref_source=faker (default)

In [23]:
from new_oop import Deidentifier
deid_implementor= Deidentifier(spark)

In [24]:
res= deid_implementor.deidentify(input_file_path="/home/jovyan/work/shared/ahmet/works/DEID_MODULE/try_data.csv",
                                output_file_path="deidentified1.csv",
                                custom_pipeline=model,
                                mode="obfuscate")


Deidentification process of 'text' field is starting...
Deidentification process of 'text' field is completed...
Deidentifcation successfully completed and the results saved as 'deidentified1.csv' !


In [25]:
res.show(truncate=False)

+---+-------------------------------+-------------------------------+
|ID |text                           |text_deidentified              |
+---+-------------------------------+-------------------------------+
|0  |He is discharged in September. |He is discharged in 06-20-1992.|
|1  |His pps number is 1234567.     |His pps number is C7912599.    |
|2  |"                              |"                              |
+---+-------------------------------+-------------------------------+



### age groups obfuscation

In [26]:
from new_oop import Deidentifier
deid_implementor= Deidentifier(spark)

In [27]:
res= deid_implementor.deidentify(input_file_path="/home/jovyan/work/shared/ahmet/works/DEID_MODULE/try_age_group.csv",
                                                output_file_path="deidentified1.csv",
                                                custom_pipeline=model,
                                                mode="obfuscate",
                                                age_group_obfuscation=True, age_ranges=[1, 4, 12, 20, 40, 60, 80])

Deidentification process of 'text' field is starting...
Deidentification process of 'text' field is completed...
Deidentifcation successfully completed and the results saved as 'deidentified1.csv' !


In [28]:
res.show(truncate=False)

+---+--------------------------------+--------------------------------+
|ID |text                            |text_deidentified               |
+---+--------------------------------+--------------------------------+
|0  |1 year old baby                 |3 year old baby                 |
|1  |4 year old kids                 |11 year old kids                |
|2  |A 15 year old female with       |A 14 year old female with       |
|3  |Record date: 2093-01-13, Age: 25|Record date: 2093-02-04, Age: 25|
|4  |Patient is 45 years-old         |Patient is 41 years-old         |
|5  |He is 65 years-old male         |He is 60 years-old male         |
+---+--------------------------------+--------------------------------+



In [30]:
import pandas as pd
pd.set_option('max_colwidth', None)
res_data= pd.read_csv("/home/jovyan/work/shared/ahmet/works/DEID_MODULE/new_version/deidentified1.csv")
res_data.head()

,ID,text,text_deidentified
0,0,1 year old baby,3 year old baby
1,1,4 year old kids,11 year old kids
2,2,A 15 year old female with,A 14 year old female with
3,3,"Record date: 2093-01-13, Age: 25","Record date: 2093-02-04, Age: 25"
4,4,Patient is 45 years-old,Patient is 41 years-old


### Shifting days according to the ID column

In [35]:
import pandas as pd
data = pd.DataFrame(
    {'patientID' : ['A001', 'A001', 'A002', 'A002'],
     'text' : ['Chris Brown was discharged on 10/02/2022', 
               'Mark White was discharged on 02/28/2020', 
               'John was discharged on 03/15/2022',
               'John Moore was discharged on 12/31/2022'
              ]
    }
)

my_input_df = spark.createDataFrame(data)

my_input_df.show(truncate = False)

+---------+----------------------------------------+
|patientID|text                                    |
+---------+----------------------------------------+
|A001     |Chris Brown was discharged on 10/02/2022|
|A001     |Mark White was discharged on 02/28/2020 |
|A002     |John was discharged on 03/15/2022       |
|A002     |John Moore was discharged on 12/31/2022 |
+---------+----------------------------------------+



In [36]:
df_pd= my_input_df.toPandas()
df_pd.to_csv("try_data.csv", index=False)

In [37]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

documentHasher = DocumentHashCoder()\
    .setInputCols("document")\
    .setOutputCol("document2")\
    .setPatientIdColumn("patientID")\
    .setRangeDays(100)\
    .setNewDateShift("shift_days")\
    .setSeed(100)

tokenizer = Tokenizer()\
    .setInputCols(["document2"])\
    .setOutputCol("token")

embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["document2", "token"])\
    .setOutputCol("word_embeddings")

clinical_ner = MedicalNerModel\
    .pretrained("ner_deid_subentity_augmented", "en", "clinical/models")\
    .setInputCols(["document2","token", "word_embeddings"])\
    .setOutputCol("ner")

ner_converter = NerConverterInternal()\
    .setInputCols(["document2", "token", "ner"])\
    .setOutputCol("ner_chunk")

#de_identification = DeIdentification() \
#    .setInputCols(["ner_chunk", "token", "document2"]) \
#    .setOutputCol("deid_text") \
#    .setMode("obfuscate") \
#    .setObfuscateDate(True) \
#    .setDateTag("DATE") \
#    .setLanguage("en") \
#    .setObfuscateRefSource('faker') \
#    .setUseShifDays(True)\
#    .setRegion('us')
    

nlpPipeline = Pipeline().setStages([
    documentAssembler,
    documentHasher,
    tokenizer,
    embeddings,
    clinical_ner,
    ner_converter])

empty_data = spark.createDataFrame([["", ""]]).toDF("text", "patientID")

pipeline_model = nlpPipeline.fit(empty_data)

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_deid_subentity_augmented download started this may take some time.
[ | ]ner_deid_subentity_augmented download started this may take some time.
Approximate size to download 14.1 MB
Download done! Loading the resource.


2023-01-27 11:02:20.032720: W external/org_tensorflow/tensorflow/core/common_runtime/colocation_graph.cc:1218] Failed to place the graph without changing the devices of some resources. Some of the operations (that had to be colocated with resource generating operations) are not supported on the resources' devices. Current candidate devices are [
  /job:localhost/replica:0/task:0/device:CPU:0].
See below for details of this colocation group:
Colocation Debug Info:
Colocation group had the following types and supported devices: 
Root Member(assigned_device_name_index_=-1 requested_device_name_='/device:GPU:0' assigned_device_name_='' resource_device_name_='/device:GPU:0' supported_device_types_=[CPU] possible_devices_=[]
ScatterAdd: CPU 
RealDiv: CPU 
VariableV2: CPU 
Identity: CPU 
Add: CPU 
AddV2: CPU 
Unique: CPU 
Shape: CPU 
Sqrt: CPU 
AssignSub: CPU 
Sub: CPU 
Const: CPU 
Fill: CPU 
RandomUniform: CPU 
Mul: CPU 
Assign: CPU 
NoOp: CPU 
GatherV2: CPU 
UnsortedSegmentSum: CPU 
Cast: C

[OK!]


In [38]:
from new_oop import Deidentifier
deid_implementor= Deidentifier(spark)

In [39]:
res= deid_implementor.deidentify(input_file_path="/home/jovyan/work/shared/ahmet/works/DEID_MODULE/new_version/try_data.csv",
                                                output_file_path="deidentified1.csv",
                                                custom_pipeline=pipeline_model,
                                                mode="obfuscate",
                                                shift_days=True,
                                                obfuscate_date=True, 
                                                ner_chunk="ner_chunk",
                                                token="token",
                                                documentHashCoder_col_name="document2",
                                                separator=",",
                                                unnormalized_date=False
                                                )

Deidentification process of 'text' field is starting...
Deidentification process of 'text' field is completed...
Deidentifcation successfully completed and the results saved as 'deidentified1.csv' !


In [40]:
res.show(truncate=False)

+---+----------------------------------------+---------------------------------------------+
|ID |text                                    |text_deidentified                            |
+---+----------------------------------------+---------------------------------------------+
|0  |Chris Brown was discharged on 10/02/2022|Jeffie Grow was discharged on 09/27/2022     |
|1  |Mark White was discharged on 02/28/2020 |Alexandria Lamas was discharged on 02/23/2020|
|2  |John was discharged on 03/15/2022       |Armin Fallen was discharged on 04/13/2022    |
|3  |John Moore was discharged on 12/31/2022 |Willian Lore was discharged on 01/29/2023    |
+---+----------------------------------------+---------------------------------------------+



### Shifting days according to specified values

In [41]:
data = pd.DataFrame(
    {'patientID' : ['A001', 'A002', 'A003', 'A004'],
     'text' : ['Chris Brown was discharged on 10/02/2022', 
               'Mark White was discharged on 03/01/2020', 
               'John was discharged on 03/15/2022',
               'John Moore was discharged on 12/31/2022'
              ],
     'dateshift' : ['10', '-2', '30', '-8']
    }
)

my_input_df = spark.createDataFrame(data)

my_input_df.show(truncate=False)

+---------+----------------------------------------+---------+
|patientID|text                                    |dateshift|
+---------+----------------------------------------+---------+
|A001     |Chris Brown was discharged on 10/02/2022|10       |
|A002     |Mark White was discharged on 03/01/2020 |-2       |
|A003     |John was discharged on 03/15/2022       |30       |
|A004     |John Moore was discharged on 12/31/2022 |-8       |
+---------+----------------------------------------+---------+



In [42]:
df_pd= my_input_df.toPandas()
df_pd.to_csv("try_data.csv", index=False)

In [43]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

documentHasher = DocumentHashCoder()\
    .setInputCols("document")\
    .setOutputCol("document2")\
    .setDateShiftColumn("dateshift")\

tokenizer = Tokenizer()\
    .setInputCols(["document2"])\
    .setOutputCol("token")

embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["document2", "token"])\
    .setOutputCol("word_embeddings")

clinical_ner = MedicalNerModel\
    .pretrained("ner_deid_subentity_augmented", "en", "clinical/models")\
    .setInputCols(["document2","token", "word_embeddings"])\
    .setOutputCol("ner")

ner_converter = NerConverterInternal()\
    .setInputCols(["document2", "token", "ner"])\
    .setOutputCol("ner_chunk")

nlpPipeline = Pipeline().setStages([
    documentAssembler,
    documentHasher,
    tokenizer,
    embeddings,
    clinical_ner,
    ner_converter
    
])

empty_data = spark.createDataFrame([["", "", ""]]).toDF("patientID","text", "dateshift")

pipeline_col_model = nlpPipeline.fit(empty_data)

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_deid_subentity_augmented download started this may take some time.
[OK!]


In [44]:
from new_oop import Deidentifier
deid_implementor= Deidentifier(spark)

In [45]:
res= deid_implementor.deidentify(input_file_path="/home/jovyan/work/shared/ahmet/works/DEID_MODULE/new_version/try_data.csv",
                                                separator=",",
                                                output_file_path="deidentified1.csv",
                                                custom_pipeline=pipeline_col_model,
                                                mode="obfuscate",
                                                shift_days=True,
                                                obfuscate_date=True, 
                                                ner_chunk="ner_chunk",
                                                token="token",
                                                documentHashCoder_col_name="document2")

Deidentification process of 'text' field is starting...
Deidentification process of 'text' field is completed...
Deidentifcation successfully completed and the results saved as 'deidentified1.csv' !


In [46]:
res.show(truncate=False)

+---+----------------------------------------+-------------------------------------------------+
|ID |text                                    |text_deidentified                                |
+---+----------------------------------------+-------------------------------------------------+
|0  |Chris Brown was discharged on 10/02/2022|Dolphus Began was discharged on 10/12/2022       |
|1  |Mark White was discharged on 03/01/2020 |Juliette Guadalajara was discharged on 02/28/2020|
|2  |John was discharged on 03/15/2022       |Howie Nyhan was discharged on 04/14/2022         |
|3  |John Moore was discharged on 12/31/2022 |Burman Hawk was discharged on 12/23/2022         |
+---+----------------------------------------+-------------------------------------------------+



### Masking Unnormalized Date Formats

In [47]:
import pandas as pd

data = pd.DataFrame(
    {'patientID' : ['A001', 'A001', 'A002', 'A002'],
     'text' : ['Chris Brown was discharged on 10/02/2022', 
               'Mark White was discharged on 02/28/2020', 
               'John was discharged on 03Apr2022',          # check this
               'John Moore was discharged on 12/31/2022'
              ],
     'dateshift' : ['-5', '-2', '10', '20']
    }
)


my_input_df = spark.createDataFrame(data)

my_input_df.show(truncate=False)

+---------+----------------------------------------+---------+
|patientID|text                                    |dateshift|
+---------+----------------------------------------+---------+
|A001     |Chris Brown was discharged on 10/02/2022|-5       |
|A001     |Mark White was discharged on 02/28/2020 |-2       |
|A002     |John was discharged on 03Apr2022        |10       |
|A002     |John Moore was discharged on 12/31/2022 |20       |
+---------+----------------------------------------+---------+



In [48]:
df_pd= my_input_df.toPandas()
df_pd.to_csv("try_data.csv", index=False)

In [49]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

documentHasher = DocumentHashCoder()\
    .setInputCols("document")\
    .setOutputCol("document2")\
    .setDateShiftColumn("dateshift")\

tokenizer = Tokenizer()\
    .setInputCols(["document2"])\
    .setOutputCol("token")

embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["document2", "token"])\
    .setOutputCol("word_embeddings")

clinical_ner = MedicalNerModel\
    .pretrained("ner_deid_subentity_augmented", "en", "clinical/models")\
    .setInputCols(["document2","token", "word_embeddings"])\
    .setOutputCol("ner")

ner_converter = NerConverterInternal()\
    .setInputCols(["document2", "token", "ner"])\
    .setOutputCol("ner_chunk")

nlpPipeline = Pipeline().setStages([
    documentAssembler,
    documentHasher,
    tokenizer,
    embeddings,
    clinical_ner,
    ner_converter    
])

empty_data = spark.createDataFrame([["", "", ""]]).toDF("patientID","text", "dateshift")

model = nlpPipeline.fit(empty_data)

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_deid_subentity_augmented download started this may take some time.
[OK!]


In [50]:
res= deid_implementor.deidentify(input_file_path="/home/jovyan/work/shared/ahmet/works/DEID_MODULE/new_version/try_data.csv",
                                                output_file_path="deidentified1.csv",
                                                custom_pipeline=model,
                                                mode="obfuscate",
                                                shift_days=True,
                                                obfuscate_date=True, 
                                                ner_chunk="ner_chunk",
                                                token="token",
                                                documentHashCoder_col_name="document2",
                                                separator=",",
                                                unnormalized_date=True,
                                                unnormalized_mode="mask"
                                                )

Deidentification process of 'text' field is starting...
Deidentification process of 'text' field is completed...
Deidentifcation successfully completed and the results saved as 'deidentified1.csv' !


In [51]:
res.show(truncate=False)

+---+----------------------------------------+---------------------------------------------+
|ID |text                                    |text_deidentified                            |
+---+----------------------------------------+---------------------------------------------+
|0  |Chris Brown was discharged on 10/02/2022|Arthur Everts was discharged on 09/27/2022   |
|1  |Mark White was discharged on 02/28/2020 |Clarene Downs was discharged on 02/26/2020   |
|2  |John was discharged on 03Apr2022        |Alanda Dennis was discharged on <DATE>       |
|3  |John Moore was discharged on 12/31/2022 |Sherryll Simmers was discharged on 01/20/2023|
+---+----------------------------------------+---------------------------------------------+



**unnormalized_mode="obfuscate"**

In [52]:
res= deid_implementor.deidentify(input_file_path="/home/jovyan/work/shared/ahmet/works/DEID_MODULE/new_version/try_data.csv",
                                                output_file_path="deidentified1.csv",
                                                custom_pipeline=model,
                                                mode="obfuscate",
                                                shift_days=True,
                                                obfuscate_date=True, 
                                                ner_chunk="ner_chunk",
                                                token="token",
                                                documentHashCoder_col_name="document2",
                                                separator=",",
                                                unnormalized_date=True,
                                                unnormalized_mode="obfuscate"
                                                )

Deidentification process of 'text' field is starting...
Deidentification process of 'text' field is completed...
Deidentifcation successfully completed and the results saved as 'deidentified1.csv' !


In [53]:
res.show(truncate=False)

+---+----------------------------------------+------------------------------------------------+
|ID |text                                    |text_deidentified                               |
+---+----------------------------------------+------------------------------------------------+
|0  |Chris Brown was discharged on 10/02/2022|Pola Koller was discharged on 09/27/2022        |
|1  |Mark White was discharged on 02/28/2020 |Jule Kugel was discharged on 02/26/2020         |
|2  |John was discharged on 03Apr2022        |Jennelle Hutchinson was discharged on 05-10-1985|
|3  |John Moore was discharged on 12/31/2022 |Ellie Snuffer was discharged on 01/20/2023      |
+---+----------------------------------------+------------------------------------------------+



# PretrainedPipeline

### Mask options

**Default: entity_labels**

In [54]:
from new_oop import Deidentifier
deid_implementor= Deidentifier(spark)

In [55]:
res= deid_implementor.deidentify(input_file_path="/home/jovyan/work/shared/ahmet/works/DEID_MODULE/try_data.csv",
                                output_file_path="deidentified1.csv",
                                mode="mask")

clinical_deidentification download started this may take some time.
Approx size to download 1.6 GB
[OK!]
Deidentification process of 'text' field is starting...
deidentification process of 'text' field is completed...
Deidentifcation successfully completed and the results saved as 'deidentified1.csv' !


In [6]:
res.show(truncate=30)

+---+------------------------------+------------------------------+
| ID|                 text_original|                     text_deid|
+---+------------------------------+------------------------------+
|  0|                             "|                           ["]|
|  1|Record date : 2093-01-13 , ...|[Record date : <DATE> , <DO...|
|  2|                             "|                           ["]|
+---+------------------------------+------------------------------+



#### fixed_length_chars

In [56]:
from new_oop import Deidentifier
deid_implementor= Deidentifier(spark)

In [57]:
res= deid_implementor.deidentify(input_file_path="/home/jovyan/work/shared/ahmet/works/DEID_MODULE/try_data.csv",
                                output_file_path="deidentified1.csv",
                                mode="mask",
                                masking_policy="fixed_length_chars",
                                fixed_mask_length=2)

clinical_deidentification download started this may take some time.
Approx size to download 1.6 GB
[OK!]
Deidentification process of 'text' field is starting...
deidentification process of 'text' field is completed...
Deidentifcation successfully completed and the results saved as 'deidentified1.csv' !


In [58]:
res.show(truncate=False)

+---+-------------------------------+-------------------------+
|ID |text_original                  |text_deid                |
+---+-------------------------------+-------------------------+
|0  |He is discharged in September. |[He is discharged in **.]|
|1  |His pps number is 1234567.     |[His pps number is **.]  |
|2  |"                              |["]                      |
+---+-------------------------------+-------------------------+



#### same_length_chars

In [59]:
from new_oop import Deidentifier
deid_implementor= Deidentifier(spark)

In [60]:
res= deid_implementor.deidentify(input_file_path="/home/jovyan/work/shared/ahmet/works/DEID_MODULE/try_data.csv",
                                output_file_path="deidentified1.csv",
                                mode="mask",
                                masking_policy="same_length_chars")

clinical_deidentification download started this may take some time.
Approx size to download 1.6 GB
[OK!]
Deidentification process of 'text' field is starting...
deidentification process of 'text' field is completed...
Deidentifcation successfully completed and the results saved as 'deidentified1.csv' !


In [61]:
res.show(truncate=False)

+---+-------------------------------+--------------------------------+
|ID |text_original                  |text_deid                       |
+---+-------------------------------+--------------------------------+
|0  |He is discharged in September. |[He is discharged in [*******].]|
|1  |His pps number is 1234567.     |[His pps number is [*****].]    |
|2  |"                              |["]                             |
+---+-------------------------------+--------------------------------+



### obfuscate options

In [62]:
text ='''
Record date : 2093-01-13 , David Hale , M.D . , Name : Hendrickson , Ora MR # 7194334 Date : 01/13/93 . Patient : Oliveira, 25 years-old , Record date : 2079-11-09 . Cocke County Baptist Hospital . 0295 Keats Street
'''
spark_df= spark.createDataFrame([[text]]).toDF("text")

df_pd= spark_df.toPandas()
df_pd.to_csv('/home/jovyan/work/shared/ahmet/works/DEID_MODULE/try_data.csv', index=False)
df_pd.head()

,text
0,"\nRecord date : 2093-01-13 , David Hale , M.D . , Name : Hendrickson , Ora MR # 7194334 Date : 01/13/93 . Patient : Oliveira, 25 years-old , Record date : 2079-11-09 . Cocke County Baptist Hospital . 0295 Keats Street\n"


In [63]:
from new_oop import Deidentifier
deid_implementor= Deidentifier(spark)

In [64]:
res= deid_implementor.deidentify(input_file_path="/home/jovyan/work/shared/ahmet/works/DEID_MODULE/try_data.csv",
                                output_file_path="deidentified1.csv",
                                separator="\t",
                                mode="obfuscate",
#                                obfuscate_ref_source="both",
                                unnormalized_date=False)
#                                obfuscate_ref_file_path="/home/jovyan/work/shared/ahmet/works/DEID_MODULE/obfuscation.txt")

clinical_deidentification download started this may take some time.
Approx size to download 1.6 GB
[OK!]
Deidentification process of 'text' field is starting...
deidentification process of 'text' field is completed...
Deidentifcation successfully completed and the results saved as 'deidentified1.csv' !


In [65]:
res.show(truncate=50) 

+---+--------------------------------------------------+--------------------------------------------------+
| ID|                                     text_original|                                         text_deid|
+---+--------------------------------------------------+--------------------------------------------------+
|  0|                                                 "|                                               ["]|
|  1|Record date : 2093-01-13 , David Hale , M.D . ,...|[Record date : 2093-01-22 , Dr Pat Bodily , M.D...|
|  2|                                                 "|                                               ["]|
+---+--------------------------------------------------+--------------------------------------------------+



#### age groups obfuscation

In [66]:
from new_oop import Deidentifier
deid_implementor= Deidentifier(spark)

In [67]:
res= deid_implementor.deidentify(input_file_path="/home/jovyan/work/shared/ahmet/works/DEID_MODULE/try_age_group.csv",
                                                output_file_path="deidentified1.csv",
                                                mode="obfuscate",
                                                age_group_obfuscation=True, age_ranges=[1, 4, 12, 20, 40, 60, 80])

clinical_deidentification download started this may take some time.
Approx size to download 1.6 GB
[OK!]
Deidentification process of 'text' field is starting...
deidentification process of 'text' field is completed...
Deidentifcation successfully completed and the results saved as 'deidentified1.csv' !


In [68]:
res.show()

+---+--------------------+--------------------+
| ID|       text_original|           text_deid|
+---+--------------------+--------------------+
|  0|     1 year old baby|    [3year old baby]|
|  1|     4 year old kids|    [8year old kids]|
|  2|A 15 year old fem...|[A 16year old fem...|
|  3|Record date: 2093...|[Record date: 209...|
|  4|Patient is 45 yea...|[Patient is 40yea...|
|  5|He is 65 years-ol...|[He is 67years-ol...|
+---+--------------------+--------------------+



#### Shifting Days

In [69]:
data = pd.DataFrame(
    {'patientID' : ['A001', 'A002', 'A003', 'A004'],
     'text' : ['Chris Brown was discharged on 10/02/2022', 
               'Mark White was discharged on 03/01/2020', 
               'John was discharged on 03/15/2022',
               'John Moore was discharged on 12/31/2022'
              ]
    }
)

my_input_df = spark.createDataFrame(data)
df_pd= my_input_df.toPandas()
df_pd.to_csv("try_data.csv", index=False)

my_input_df.show(truncate=False)

+---------+----------------------------------------+
|patientID|text                                    |
+---------+----------------------------------------+
|A001     |Chris Brown was discharged on 10/02/2022|
|A002     |Mark White was discharged on 03/01/2020 |
|A003     |John was discharged on 03/15/2022       |
|A004     |John Moore was discharged on 12/31/2022 |
+---------+----------------------------------------+



In [70]:
from new_oop import Deidentifier
deid_implementor= Deidentifier(spark)

In [71]:
res= deid_implementor.deidentify(input_file_path="/home/jovyan/work/shared/ahmet/works/DEID_MODULE/new_version/try_data.csv",
                                                output_file_path="deidentified1.csv",
                                                mode="obfuscate",
                                                shift_days=True,
                                                obfuscate_date=True, 
                                                separator=",",
                                                number_of_days=1
                                                )

clinical_deidentification download started this may take some time.
Approx size to download 1.6 GB
[OK!]
Deidentification process of 'text' field is starting...
deidentification process of 'text' field is completed...
Deidentifcation successfully completed and the results saved as 'deidentified1.csv' !


In [73]:
res.show(truncate=False)

+---+----------------------------------------+--------------------------------------------+
|ID |text_original                           |text_deid                                   |
+---+----------------------------------------+--------------------------------------------+
|0  |Chris Brown was discharged on 10/02/2022|[Dotty Neat was discharged on 10/03/2022]   |
|1  |Mark White was discharged on 03/01/2020 |[Jennings Abt was discharged on 03/02/2020] |
|2  |John was discharged on 03/15/2022       |[Bolling Kells was discharged on 03/16/2022]|
|3  |John Moore was discharged on 12/31/2022 |[Ellender Rom was discharged on 01/01/2023] |
+---+----------------------------------------+--------------------------------------------+



In [2]:
from new_oop import Deidentifier
deid_implementor= Deidentifier(spark)

In [3]:
res= deid_implementor.deidentify(input_file_path="/home/jovyan/work/shared/ahmet/works/DEID_MODULE/new_version/try_data.csv",
                                                output_file_path="deidentified1.csv",
                                                mode="obfuscate",
                                                shift_days=True,
                                                obfuscate_date=True, 
                                                id_column_name="patientID",
                                                separator=",",
                                                unnormalized_date=False
                                                )

clinical_deidentification download started this may take some time.
Approx size to download 1.6 GB
[ | ]clinical_deidentification download started this may take some time.
Approximate size to download 1.6 GB
Download done! Loading the resource.
[ / ]

2023-01-26 12:17:32.026893: I external/org_tensorflow/tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


[ | ]

2023-01-26 12:17:38.168331: W external/org_tensorflow/tensorflow/core/common_runtime/colocation_graph.cc:1218] Failed to place the graph without changing the devices of some resources. Some of the operations (that had to be colocated with resource generating operations) are not supported on the resources' devices. Current candidate devices are [
  /job:localhost/replica:0/task:0/device:CPU:0].
See below for details of this colocation group:
Colocation Debug Info:
Colocation group had the following types and supported devices: 
Root Member(assigned_device_name_index_=-1 requested_device_name_='/device:GPU:0' assigned_device_name_='' resource_device_name_='/device:GPU:0' supported_device_types_=[CPU] possible_devices_=[]
ScatterAdd: CPU 
RealDiv: CPU 
VariableV2: CPU 
Identity: CPU 
Add: CPU 
AddV2: CPU 
Unique: CPU 
Shape: CPU 
Sqrt: CPU 
AssignSub: CPU 
Sub: CPU 
Const: CPU 
Fill: CPU 
RandomUniform: CPU 
Mul: CPU 
Assign: CPU 
NoOp: CPU 
GatherV2: CPU 
UnsortedSegmentSum: CPU 
Cast: C

[ / ]

2023-01-26 12:17:39.883475: W external/org_tensorflow/tensorflow/core/common_runtime/colocation_graph.cc:1218] Failed to place the graph without changing the devices of some resources. Some of the operations (that had to be colocated with resource generating operations) are not supported on the resources' devices. Current candidate devices are [
  /job:localhost/replica:0/task:0/device:CPU:0].
See below for details of this colocation group:
Colocation Debug Info:
Colocation group had the following types and supported devices: 
Root Member(assigned_device_name_index_=-1 requested_device_name_='/device:GPU:0' assigned_device_name_='' resource_device_name_='/device:GPU:0' supported_device_types_=[CPU] possible_devices_=[]
ScatterAdd: CPU 
RealDiv: CPU 
VariableV2: CPU 
Identity: CPU 
Add: CPU 
AddV2: CPU 
Unique: CPU 
Shape: CPU 
Sqrt: CPU 
AssignSub: CPU 
Sub: CPU 
Const: CPU 
Fill: CPU 
RandomUniform: CPU 
Mul: CPU 
Assign: CPU 
NoOp: CPU 
GatherV2: CPU 
UnsortedSegmentSum: CPU 
Cast: C

[OK!]


Deidentifcation successfully completed and the results saved as 'deidentified1.csv' !


In [5]:
res.show(truncate=False)

+---+----------------------------------------+--------------------------------------------+
|ID |text_original                           |text_deid                                   |
+---+----------------------------------------+--------------------------------------------+
|0  |Chris Brown was discharged on 10/02/2022|[Dotty Neat was discharged on 26/03/2022]   |
|1  |Mark White was discharged on 02/28/2020 |[Jennings Abt was discharged on 04-09-1981] |
|2  |John was discharged on 03Apr2022        |[Bolling Kells was discharged on 11-02-2001]|
|3  |John Moore was discharged on 12/31/2022 |[Ellender Rom was discharged on 12-08-1988] |
+---+----------------------------------------+--------------------------------------------+



In [2]:
data = pd.DataFrame(
    {'patientID' : ['A001', 'A002', 'A003', 'A004'],
     'text' : ['Chris Brown was discharged on 10/02/2022', 
               'Mark White was discharged on 03/01/2020', 
               'John was discharged on 03/15/2022',
               'John Moore was discharged on 12/31/2022'
              ],
     'dateshift' : ['10', '-2', '30', '-8']
    }
)

my_input_df = spark.createDataFrame(data)

my_input_df.show(truncate=False)

+---------+----------------------------------------+---------+
|patientID|text                                    |dateshift|
+---------+----------------------------------------+---------+
|A001     |Chris Brown was discharged on 10/02/2022|10       |
|A002     |Mark White was discharged on 03/01/2020 |-2       |
|A003     |John was discharged on 03/15/2022       |30       |
|A004     |John Moore was discharged on 12/31/2022 |-8       |
+---------+----------------------------------------+---------+



In [3]:
df_pd= my_input_df.toPandas()
df_pd.to_csv("try_data.csv", index=False)

In [2]:
from new_oop import Deidentifier
deid_implementor= Deidentifier(spark)

In [3]:
res= deid_implementor.deidentify(input_file_path="/home/jovyan/work/shared/ahmet/works/DEID_MODULE/new_version/try_data.csv",
                                                separator=",",
                                                output_file_path="deidentified1.csv",
                                                mode="obfuscate",
                                                shift_days=True,
                                                obfuscate_date=True,
                                                date_shift_column_name="dateshift"
                                                )

clinical_deidentification download started this may take some time.
Approx size to download 1.6 GB
[ | ]clinical_deidentification download started this may take some time.
Approximate size to download 1.6 GB
Download done! Loading the resource.
[ / ]

2023-01-26 13:53:19.718874: I external/org_tensorflow/tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


[ | ]

2023-01-26 13:53:25.951308: W external/org_tensorflow/tensorflow/core/common_runtime/colocation_graph.cc:1218] Failed to place the graph without changing the devices of some resources. Some of the operations (that had to be colocated with resource generating operations) are not supported on the resources' devices. Current candidate devices are [
  /job:localhost/replica:0/task:0/device:CPU:0].
See below for details of this colocation group:
Colocation Debug Info:
Colocation group had the following types and supported devices: 
Root Member(assigned_device_name_index_=-1 requested_device_name_='/device:GPU:0' assigned_device_name_='' resource_device_name_='/device:GPU:0' supported_device_types_=[CPU] possible_devices_=[]
ScatterAdd: CPU 
RealDiv: CPU 
VariableV2: CPU 
Identity: CPU 
Add: CPU 
AddV2: CPU 
Unique: CPU 
Shape: CPU 
Sqrt: CPU 
AssignSub: CPU 
Sub: CPU 
Const: CPU 
Fill: CPU 
RandomUniform: CPU 
Mul: CPU 
Assign: CPU 
NoOp: CPU 
GatherV2: CPU 
UnsortedSegmentSum: CPU 
Cast: C

[ / ]

2023-01-26 13:53:27.752087: W external/org_tensorflow/tensorflow/core/common_runtime/colocation_graph.cc:1218] Failed to place the graph without changing the devices of some resources. Some of the operations (that had to be colocated with resource generating operations) are not supported on the resources' devices. Current candidate devices are [
  /job:localhost/replica:0/task:0/device:CPU:0].
See below for details of this colocation group:
Colocation Debug Info:
Colocation group had the following types and supported devices: 
Root Member(assigned_device_name_index_=-1 requested_device_name_='/device:GPU:0' assigned_device_name_='' resource_device_name_='/device:GPU:0' supported_device_types_=[CPU] possible_devices_=[]
ScatterAdd: CPU 
RealDiv: CPU 
VariableV2: CPU 
Identity: CPU 
Add: CPU 
AddV2: CPU 
Unique: CPU 
Shape: CPU 
Sqrt: CPU 
AssignSub: CPU 
Sub: CPU 
Const: CPU 
Fill: CPU 
RandomUniform: CPU 
Mul: CPU 
Assign: CPU 
NoOp: CPU 
GatherV2: CPU 
UnsortedSegmentSum: CPU 
Cast: C

[OK!]


Deidentifcation successfully completed and the results saved as 'deidentified1.csv' !


In [4]:
res.head()

,ID,text_original,text_deid
0,0,Chris Brown was discharged on 10/02/2022,[Dotty Neat was discharged on 23/02/2022]
1,1,Mark White was discharged on 03/01/2020,[Jennings Abt was discharged on 03/02/2020]
2,2,John was discharged on 03/15/2022,[Bolling Kells was discharged on 01-26-1998]
3,3,John Moore was discharged on 12/31/2022,[Ellender Rom was discharged on 12-08-1988]


#### Masking Unnormalized Date Formats

In [75]:
import pandas as pd

data = pd.DataFrame(
    {'patientID' : ['A001', 'A001', 'A002', 'A002'],
     'text' : ['Chris Brown was discharged on 10/02/2022', 
               'Mark White was discharged on 02/28/2020', 
               'John was discharged on 03Apr2022',          # check this
               'John Moore was discharged on 12/31/2022'
              ]
    }
)


my_input_df = spark.createDataFrame(data)

my_input_df.show(truncate=False)

+---------+----------------------------------------+
|patientID|text                                    |
+---------+----------------------------------------+
|A001     |Chris Brown was discharged on 10/02/2022|
|A001     |Mark White was discharged on 02/28/2020 |
|A002     |John was discharged on 03Apr2022        |
|A002     |John Moore was discharged on 12/31/2022 |
+---------+----------------------------------------+



In [76]:
df_pd= my_input_df.toPandas()
df_pd.to_csv("try_data.csv", index=False)

In [2]:
from new_oop import Deidentifier
deid_implementor= Deidentifier(spark)

In [3]:
res= deid_implementor.deidentify(input_file_path="/home/jovyan/work/shared/ahmet/works/DEID_MODULE/new_version/try_data.csv",
                                                output_file_path="deidentified1.csv",
                                                mode="obfuscate",
                                                shift_days=True,
                                                obfuscate_date=True, 
                                                separator=",",
                                                unnormalized_date=True,
                                                unnormalized_mode="mask"
                                                )

clinical_deidentification download started this may take some time.
Approx size to download 1.6 GB
[ | ]clinical_deidentification download started this may take some time.
Approximate size to download 1.6 GB
Download done! Loading the resource.
[ / ]

2023-01-27 11:17:49.213568: I external/org_tensorflow/tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


[ — ]

2023-01-27 11:18:01.594610: W external/org_tensorflow/tensorflow/core/common_runtime/colocation_graph.cc:1218] Failed to place the graph without changing the devices of some resources. Some of the operations (that had to be colocated with resource generating operations) are not supported on the resources' devices. Current candidate devices are [
  /job:localhost/replica:0/task:0/device:CPU:0].
See below for details of this colocation group:
Colocation Debug Info:
Colocation group had the following types and supported devices: 
Root Member(assigned_device_name_index_=-1 requested_device_name_='/device:GPU:0' assigned_device_name_='' resource_device_name_='/device:GPU:0' supported_device_types_=[CPU] possible_devices_=[]
ScatterAdd: CPU 
RealDiv: CPU 
VariableV2: CPU 
Identity: CPU 
Add: CPU 
AddV2: CPU 
Unique: CPU 
Shape: CPU 
Sqrt: CPU 
AssignSub: CPU 
Sub: CPU 
Const: CPU 
Fill: CPU 
RandomUniform: CPU 
Mul: CPU 
Assign: CPU 
NoOp: CPU 
GatherV2: CPU 
UnsortedSegmentSum: CPU 
Cast: C

[ \ ]

2023-01-27 11:18:03.296866: W external/org_tensorflow/tensorflow/core/common_runtime/colocation_graph.cc:1218] Failed to place the graph without changing the devices of some resources. Some of the operations (that had to be colocated with resource generating operations) are not supported on the resources' devices. Current candidate devices are [
  /job:localhost/replica:0/task:0/device:CPU:0].
See below for details of this colocation group:
Colocation Debug Info:
Colocation group had the following types and supported devices: 
Root Member(assigned_device_name_index_=-1 requested_device_name_='/device:GPU:0' assigned_device_name_='' resource_device_name_='/device:GPU:0' supported_device_types_=[CPU] possible_devices_=[]
ScatterAdd: CPU 
RealDiv: CPU 
VariableV2: CPU 
Identity: CPU 
Add: CPU 
AddV2: CPU 
Unique: CPU 
Shape: CPU 
Sqrt: CPU 
AssignSub: CPU 
Sub: CPU 
Const: CPU 
Fill: CPU 
RandomUniform: CPU 
Mul: CPU 
Assign: CPU 
NoOp: CPU 
GatherV2: CPU 
UnsortedSegmentSum: CPU 
Cast: C

[OK!]
Deidentification process of 'text' field is starting...


deidentification process of 'text' field is completed...
Deidentifcation successfully completed and the results saved as 'deidentified1.csv' !


In [4]:
res.show(truncate=False)

+---+----------------------------------------+-------------------------------------------+
|ID |text_original                           |text_deid                                  |
+---+----------------------------------------+-------------------------------------------+
|0  |Chris Brown was discharged on 10/02/2022|[Dotty Neat was discharged on 11/07/2022]  |
|1  |Mark White was discharged on 02/28/2020 |[Jennings Abt was discharged on 03/28/2020]|
|2  |John was discharged on 03Apr2022        |[Bolling Kells was discharged on <DATE>]   |
|3  |John Moore was discharged on 12/31/2022 |[Ellender Rom was discharged on 02/17/2023]|
+---+----------------------------------------+-------------------------------------------+



In [5]:
res= deid_implementor.deidentify(input_file_path="/home/jovyan/work/shared/ahmet/works/DEID_MODULE/new_version/try_data.csv",
                                                output_file_path="deidentified1.csv",
                                                mode="obfuscate",
                                                shift_days=True,
                                                obfuscate_date=True, 
                                                separator=",",
                                                unnormalized_date=True,
                                                unnormalized_mode="obfuscate"
                                                )

clinical_deidentification download started this may take some time.
Approx size to download 1.6 GB
[OK!]
Deidentification process of 'text' field is starting...
deidentification process of 'text' field is completed...
Deidentifcation successfully completed and the results saved as 'deidentified1.csv' !


In [6]:
res.show(truncate=False)

+---+----------------------------------------+--------------------------------------------+
|ID |text_original                           |text_deid                                   |
+---+----------------------------------------+--------------------------------------------+
|0  |Chris Brown was discharged on 10/02/2022|[Dotty Neat was discharged on 11/04/2022]   |
|1  |Mark White was discharged on 02/28/2020 |[Jennings Abt was discharged on 04/06/2020] |
|2  |John was discharged on 03Apr2022        |[Bolling Kells was discharged on 11-02-2001]|
|3  |John Moore was discharged on 12/31/2022 |[Ellender Rom was discharged on 02/09/2023] |
+---+----------------------------------------+--------------------------------------------+



In [3]:
from new_oop import Deidentifier
deid_implementor= Deidentifier(spark)

In [4]:
res= deid_implementor.deidentify(input_file_path="/home/jovyan/work/shared/ahmet/works/DEID_MODULE/try_age_group.csv",
                                output_file_path="deidentified1.csv",
                                mode="obfuscate",
                                age_group_obfuscation=True, age_ranges=[1, 4, 12, 20, 40, 60, 80])

clinical_deidentification download started this may take some time.
Approx size to download 1.6 GB
[ | ]clinical_deidentification download started this may take some time.
Approximate size to download 1.6 GB
Download done! Loading the resource.
[ / ]

2023-01-24 23:27:30.266742: I external/org_tensorflow/tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


[ — ]

2023-01-24 23:27:41.881379: W external/org_tensorflow/tensorflow/core/common_runtime/colocation_graph.cc:1218] Failed to place the graph without changing the devices of some resources. Some of the operations (that had to be colocated with resource generating operations) are not supported on the resources' devices. Current candidate devices are [
  /job:localhost/replica:0/task:0/device:CPU:0].
See below for details of this colocation group:
Colocation Debug Info:
Colocation group had the following types and supported devices: 
Root Member(assigned_device_name_index_=-1 requested_device_name_='/device:GPU:0' assigned_device_name_='' resource_device_name_='/device:GPU:0' supported_device_types_=[CPU] possible_devices_=[]
ScatterAdd: CPU 
RealDiv: CPU 
VariableV2: CPU 
Identity: CPU 
Add: CPU 
AddV2: CPU 
Unique: CPU 
Shape: CPU 
Sqrt: CPU 
AssignSub: CPU 
Sub: CPU 
Const: CPU 
Fill: CPU 
RandomUniform: CPU 
Mul: CPU 
Assign: CPU 
NoOp: CPU 
GatherV2: CPU 
UnsortedSegmentSum: CPU 
Cast: C

[ \ ]

2023-01-24 23:27:43.700770: W external/org_tensorflow/tensorflow/core/common_runtime/colocation_graph.cc:1218] Failed to place the graph without changing the devices of some resources. Some of the operations (that had to be colocated with resource generating operations) are not supported on the resources' devices. Current candidate devices are [
  /job:localhost/replica:0/task:0/device:CPU:0].
See below for details of this colocation group:
Colocation Debug Info:
Colocation group had the following types and supported devices: 
Root Member(assigned_device_name_index_=-1 requested_device_name_='/device:GPU:0' assigned_device_name_='' resource_device_name_='/device:GPU:0' supported_device_types_=[CPU] possible_devices_=[]
ScatterAdd: CPU 
RealDiv: CPU 
VariableV2: CPU 
Identity: CPU 
Add: CPU 
AddV2: CPU 
Unique: CPU 
Shape: CPU 
Sqrt: CPU 
AssignSub: CPU 
Sub: CPU 
Const: CPU 
Fill: CPU 
RandomUniform: CPU 
Mul: CPU 
Assign: CPU 
NoOp: CPU 
GatherV2: CPU 
UnsortedSegmentSum: CPU 
Cast: C

[OK!]


Deidentifcation successfully completed and the results saved as 'deidentified1.csv' !


In [6]:
res.show(truncate=False)

+---+--------------------------------+----------------------------------+
|ID |text_original                   |text_deid                         |
+---+--------------------------------+----------------------------------+
|0  |1 year old baby                 |[3year old baby]                  |
|1  |4 year old kids                 |[8year old kids]                  |
|2  |A 15 year old female with       |[A 16year old female with]        |
|3  |Record date: 2093-01-13, Age: 25|[Record date: 2093-02-23, Age: 35]|
|4  |Patient is 45 years-old         |[Patient is 40years-old]          |
|5  |He is 65 years-old male         |[He is 60years-old male]          |
+---+--------------------------------+----------------------------------+



# Structured Deidentification

In [7]:
input_path= "/home/jovyan/work/shared/ahmet/works/DEID_MODULE/try_structured_df.csv"

## With all default values

In [8]:
from new_oop import Deidentifier
deid_implementor= Deidentifier(spark)

In [9]:
res= deid_implementor.structured_deidentifier(input_file_path= input_path)

Deidentifcation successfully completed and the results saved as 'deidentified.csv' !


In [10]:
res.show()

+-------------------+----------+----+--------------------+-------+--------------+---+---+
|               NAME|       DOB| AGE|             ADDRESS|ZIPCODE|           TEL|SBP|DBP|
+-------------------+----------+----+--------------------+-------+--------------+---+---+
|      [Katrina Gey]|04/02/1935|[60]|711-2880 Nulla St...|  69200|(257) 563-7401|101| 42|
|   [Hershel Hidden]|03/10/2009| [7]|P.O. Box 283 8562...|  20620|(372) 587-2335|159|122|
|   [Madonna Conger]|11/01/1921|[60]|5543 Aliquet St. ...|  20783|(717) 450-4729|149| 52|
|  [Arminda Chester]|13/02/2002|[16]|Ap #867-859 Sit R...|  39531|(793) 151-6230|134|115|
|     [Norine Chars]|20/08/1942|[60]|7292 Dictum Av. S...|  47096|(492) 709-6392|139| 78|
|      [Andris Oman]|12/05/1973|[51]|Ap #651-8679 Soda...|  10855|(654) 393-5734|120|112|
|       [Cy Gunning]|11/01/1991|[23]|191-103 Integer R...|   8219|(404) 960-3807|143|126|
|       [Nelson Lim]|18/11/1937|[60]|P.O. Box 887 2508...|  12482|(314) 244-6306|147| 75|
|  [Trey S

## refsource=File

In [11]:
ref_file= "/home/jovyan/work/shared/ahmet/works/DEID_MODULE/obfuscator_unique_ref_test.txt"

In [12]:
res= deid_implementor.structured_deidentifier(input_file_path= "/home/jovyan/work/shared/ahmet/works/DEID_MODULE/try_structured_df.csv",
                                              ref_source="file", 
                                              columns_dict={'NAME': 'PATIENT', 'AGE': 'AGE'},
                                              columns_seed={"NAME": 23, "AGE": 23}, 
                                              obfuscateRefFile=ref_file)

Deidentifcation successfully completed and the results saved as 'deidentified.csv' !


In [13]:
res.show()

+------------------+----------+----+--------------------+-------+--------------+---+---+
|              NAME|       DOB| AGE|             ADDRESS|ZIPCODE|           TEL|SBP|DBP|
+------------------+----------+----+--------------------+-------+--------------+---+---+
|   [Inci FOUNTAIN]|04/02/1935|[60]|711-2880 Nulla St...|  69200|(257) 563-7401|101| 42|
|     [Jack SANTOS]|03/10/2009|[30]|P.O. Box 283 8562...|  20620|(372) 587-2335|159|122|
|[Darene GEORGIOUS]|11/01/1921|[30]|5543 Aliquet St. ...|  20783|(717) 450-4729|149| 52|
| [Shelly WELLBECK]|13/02/2002|[40]|Ap #867-859 Sit R...|  39531|(793) 151-6230|134|115|
|   [Hubert GROGAN]|20/08/1942|[40]|7292 Dictum Av. S...|  47096|(492) 709-6392|139| 78|
|      [Kalil AMIN]|12/05/1973|[40]|Ap #651-8679 Soda...|  10855|(654) 393-5734|120|112|
|  [ALTHEA COLBURN]|11/01/1991|[60]|191-103 Integer R...|   8219|(404) 960-3807|143|126|
|  [Thomson THOMAS]|18/11/1937|[60]|P.O. Box 887 2508...|  12482|(314) 244-6306|147| 75|
|     [Jack SANTOS]|1

## dayShift

In [14]:
# We can shift n days in the structured deidentification through "days" parameter when the column is a Date.

df = spark.createDataFrame([
            ["Juan García", "13/02/1977", "711 Nulla St.", "140", "673 431234"],
            ["Will Smith", "23/02/1977", "1 Green Avenue.", "140", "+23 (673) 431234"],
            ["Pedro Ximénez", "11/04/1900", "Calle del Libertador, 7", "100", "912 345623"]
        ]).toDF("NAME", "DOB", "ADDRESS", "SBP", "TEL")

df_pd= df.toPandas()
#df_pd.to_csv("try_dayshift_structured_df.csv", index=False)
df_pd.head()

,NAME,DOB,ADDRESS,SBP,TEL
0,Juan García,13/02/1977,711 Nulla St.,140,673 431234
1,Will Smith,23/02/1977,1 Green Avenue.,140,+23 (673) 431234
2,Pedro Ximénez,11/04/1900,"Calle del Libertador, 7",100,912 345623


In [15]:
res= deid_implementor.structured_deidentifier(input_file_path= "/home/jovyan/work/shared/ahmet/works/DEID_MODULE/try_dayshift_structured_df.csv",
                                              columns_dict= {"NAME": "ID", "DOB": "DATE"},
                                              columns_seed= {"NAME": 23, "DOB": 23},
                                              ref_source="faker",
                                              shift_days= 5
                                             )

Deidentifcation successfully completed and the results saved as 'deidentified.csv' !


In [16]:
res.show()

+----------+------------+--------------------+---+----------------+
|      NAME|         DOB|             ADDRESS|SBP|             TEL|
+----------+------------+--------------------+---+----------------+
|[N2649912]|[18/02/1977]|       711 Nulla St.|140|      673 431234|
| [W466004]|[28/02/1977]|     1 Green Avenue.|140|+23 (673) 431234|
| [M403810]|[16/04/1900]|Calle del Liberta...|100|      912 345623|
+----------+------------+--------------------+---+----------------+

